In [1]:
import os
print(os.environ['HF_HOME'])

/Volumes/T7/huggingface


In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("mps")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Phi-2 Inference

In [8]:
def phi_2_infer(text, max_length: int=200):
    inputs = tokenizer(text, return_tensors="pt", return_attention_mask=False)
    outputs = model.generate(**inputs, max_length=max_length)
    text = tokenizer.batch_decode(outputs)[0]
    return text

In [9]:
print(phi_2_infer(
"""
Summarize the following text:
The central clock in the human body is located in the hypothalamus, specifically in the region known as the suprachiasmatic nucleus (SCN).

The SCN takes timing input from the eyes, through photosensitive cells and pathways. The mammalian eye contains Intrinsically photosensitive retinal ganglion cells (ipRGCs), also called photosensitive retinal ganglion cells (pRGC), or melanopsin-containing retinal ganglion cells (mRGCs). As suggested by the name, these cells respond to light in an independent system from the rods and cones commonly known to be functional parts of the retina. They were discovered through experiments observing that the eyes of blind mice still respond to light, despite lack of rods and cones. 

When discussing circadian alignment and misalignment, the concepts of advancing and delaying schedules are important. 
An advancing schedule will cause the circadian clock to reach key biorhythm points, such as DLMO, core body temperature minimum, and wake time, earlier and earlier. These events appear to "advance" through the day, hence the name.

For example, if someone goes to sleep perfectly at midnight and wakes up 8 hours later each and every day (sounds nice!) changes their schedule to an advancing one, they will feel sleepy a bit earlier, and may wake up minutes or hours earlier depending on the intensity of that advancing effect. Optimal advancing schedules can move a wake time about 7 hours earlier in 3 days time. 

Said differently, the biorhythms reach key transition points closer together, in under 24 hours. In our example, the person's consistent 8:00am wake time happened every 24 hours. Advancing schedules might give a 7:30am wake time the next day, which is 23.5 hours after the last wake time. The second day of an advancing schedule could lead to a 6:45 wake time the next day, which is 23.25 hours after the previous wake-up.

Advancing schedules seek the brightest light possible over the 10 hours following the body's core body temperature minimum (CBT min).

Since CBT minima usually occur during sleep, when darkness is preferred, advancing scheduling advice often looks like "Get bright light from the time you wake up until early to mid afternoon. Avoid light as much as possible in the afternoon. Turn out the lights earlier than usual, and go to sleep when you're feeling tired enough."
Delaying schedules have the opposite effect: events occur over 24 hours apart, appear to delay throughout the day. Someone on a delaying schedule will wake up later and later each day. 

Delaying schedules will avoid light until mid-to-late afternoon, then get the brightest light possible into the night. Sleep is pushed back due to: (1) suppression of melatonin release by light (this is the importance of "D" in DLMO!) and (2) slowing down of the circadian rhythm, which causes the 2-process model that combines circadian state and homeostatic sleep pressure to reach the sleep threshold later.


People unintentionally put themselves on delaying schedules commonly in our modern world. Smart phones, tablets, televisions, and small displays of all kinds abound in our offices and homes, and these all put off light. Even with a lamp, reading before bed or in the middle of the night when sleep won't come adds to the night your brain sees at night.

""",
max_length=2000
))

KeyboardInterrupt: 

In [5]:
print(phi_2_infer("""
A: My night shift is killing me. I'm so tired. I can't sleep during the day.
Dr. B: I'm sorry to hear that. I am a doctor specializing in circadian alignment therapy. I hope we can work to find a solution together. First, let me ask: how long have you been working night shifts?
A: I've been working night shifts for about a year now.
"""))



A: My night shift is killing me. I'm so tired. I can't sleep during the day.
Dr. B: I'm sorry to hear that. I am a doctor specializing in circadian alignment therapy. I hope we can work to find a solution together. First, let me ask: how long have you been working night shifts?
A: I've been working night shifts for about a year now.
Dr. B: Okay, that's good to know. Have you noticed any changes in your sleep patterns or mood since starting the night shift?
A: Yes, I have trouble sleeping during the day and I feel more irritable and anxious.
Dr. B: I see. Well, there are a few things we can try to help you adjust to the night shift. First, we can work on creating a sleep-friendly environment during the day. This might include using blackout curtains, earplugs, and a white noise machine. We can also


In [ ]:
prompt = \
"""

"""

# Phi-2 Finetuning
Mostly following the finetuning notebook, also https://huggingface.co/docs/transformers/tasks/language_modeling

In [4]:
fine_tuned_folder = "phi_2_fine_tuned"

In [28]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir=fine_tuned_folder,
    learning_rate=2e-5,
    # per_device_train_batch_size=16,
    # per_device_eval_batch_size=16,
    use_mps_device=True,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True)

/Users/eric/Engineering/GitHub/transformers/src/transformers/training_args.py:1897: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


In [13]:

id2label = {0: "NEGATIVE", 1: "POSITIVE", -1: "UNKNOWN"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1, "UNKNOWN": -1}

In [8]:
from datasets import load_dataset

imdb = load_dataset("imdb")

In [35]:

def label_combiner(text_num_score: dict) -> str:
    label = "Reviewer: " + text_num_score["text"]
    label += "\n"
    label += "AI: " + id2label[text_num_score["label"]]

    return {"text": label}
imdb_combined = imdb.map(label_combiner)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [37]:


def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)
tokenized_imdb = imdb_combined.map(
    preprocess_function, 
    batched=True,
    num_proc=8,
    remove_columns=imdb["train"].column_names)

Map (num_proc=8):   0%|          | 0/25000 [00:00<?, ? examples/s]

/opt/homebrew/Caskroom/miniforge/base/envs/transformers/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map (num_proc=8):   0%|          | 0/25000 [00:00<?, ? examples/s]

/opt/homebrew/Caskroom/miniforge/base/envs/transformers/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map (num_proc=8):   0%|          | 0/50000 [00:00<?, ? examples/s]

/opt/homebrew/Caskroom/miniforge/base/envs/transformers/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [38]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False)

In [39]:
imdb_combined['train'][0]['text']

'Reviewer: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

In [29]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    # compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [41]:
trainer.train()

  0%|          | 0/6250 [00:00<?, ?it/s]